# Clase Práctica #2 (Compilación)

En esta clase estaremos implementando un _parser_ para el subconjunto del lenguaje `HULK` descrito en la clase anterior. Esta vez nos apoyaremos en una descripción formal del lenguaje: una gramática libre del contexto.

Recordemos que una gramática `G` es un cuádruplo `<T,N,S,P>` donde:
- `T` es el conjunto de los _terminales_ (informalmente los símbolos que realmente se imprimiran en la cadena).
- `N` es el conjunto de los _no terminales_ (símbolos intermedios usados al generar una cadena y que deberán ser reemplazados para obtener la cadena final).
- `S` es _el símbolo distinguido_ de la gramática (por definición, toda cadena perteneciente al lenguaje generado por la gramática deriva en 0 o más pasos del símbolo distinguido, o sea, `L(G) = { w | S =>* w }` ).
- `P` es el conjunto de las _producciones_ de la gramática.

## Parsing recursivo descendente

En conferencia se discutió la idea de construir un parser partiendo de la especificación de la gramática y usando una exploración con _backtrack_. Vimos que incluso algunas gramáticas podrían ser parseadas con este mecanismo sin hacer backtrack si quiera: el parser podría seleccionar de forma determinista qué producción aplicar para obtener la derivación de la cadena. A estas gramáticas les llamamos _gramáticas LL(1)_.

A continuación se presenta una implementación base del mecanismo de parsing recursivo descendente. Este facilitará la construcción _"ad hoc"_ de parsers para gramáticas específicas. En clases posteriores estaremos automatizando la generación del parser a partir de la descripción de la gramática.

In [3]:
class ParsingError(Exception):
    """
    Base class for all parsing exceptions.
    """
    pass


class Token:
    """
    Basic token class. 
    
    Parameters
    ----------
    lex : str
        Token's lexeme.
    token_type : Enum
        Token's type.
    """
    
    def __init__(self, lex, token_type):
        self.lex = lex
        self.token_type = token_type
        

class Lexer:
    """
    Base lexer class.
    
    Parameters
    ----------
    text : str
        String to tokenize.
    """
    
    def __init__(self, text):
        self.index = 0
        self.text = text
        self.tokens = self.tokenize_text()
    
    def tokenize_text(self):
        """
        Tokenize `self.text` and set it to `self.tokens`.
        """
        raise NotImplementedError()
    
    def next_token(self):
        """
        Returns the next tokens readed by the lexer. `None` if `self.tokens` is exhausted.
        """
        try:
            token = self.tokens[self.index]
            self.index += 1
            return token
        except IndexError:
            return None
    
    def is_done(self):
        """
        Returns whether or not `self.tokens` is exhausted.
        """
        try:
            self.tokens[self.index]
            return False
        except IndexError:
            return True
            

class Parser:
    """
    Base parser class.
    """
    
    def __init__(self):
        self.lexer = None
        self.left_parse = None
        self.lookahead = None
        self.lookaheadValue = None
    def parse(self, lexer):
        """
        Returns a left parse given the tokens from the lexer.
        """
        try:
            self.lexer = lexer
            self.left_parse = []
            nextToken = lexer.next_token() 
            self.lookahead = nextToken.token_type
            self.lookaheadValue = nextToken.lex
            self.begin()
            return self.left_parse
        
        except ParsingError as error:
            print(f'Parsing error: {error}!!!')
            print(f'Lookahead: {self.lookahead}')
            print(f'Unfinished parse: {self.left_parse}')
            
        finally:
            self.lex = None
            self.left_parse = None
            self.lookahead = None
            
    def begin(self):
        """
        Begin parsing from starting symbol and match EOF.
        """
        raise NotImplementedError()
        
    def report(self, production):
        """
        Adds production to the left parse that is being build.
        """
        self.left_parse.append(production)
        
    def error(self, msg=None):
        """
        Raises a parsing error.
        """
        raise ParsingError(msg)
        
    def match(self, token_type):
        """
        Consumes one token from the lexer if lookahead matches the given token type.
        Raises parsing error otherwise.
        """
        if token_type == self.lookahead:
            try:
                nextToken = self.lexer.next_token()
                self.lookahead = nextToken.token_type
                self.lookaheadValue = nextToken.lex
                # print(self.lookaheadValue)
            except AttributeError:
                self.lookahead = None
        else:
            self.error('Unexpected token')

## HULK

Comenzaremos la construcción del parser para `HULK` definiendo los tokens del lenguaje. Estos tokens representan a su vez los símbolos terminales de la gramática con la que trabajará el parser. Se incluye un token `EOF` usado para marcar el fin la cadena. En `fixed_tokens` se almacenan los tokens con lexemas constantes para simplificar la implementación del parser.

In [30]:
from enum import Enum

TokenType = Enum('TokenType', 'eof num plus minus star div opar cpar id')

EOF_TOKEN = Token('$', TokenType.eof)

fixed_tokens = {
    '+'  :   Token( '+'           , TokenType.plus  ),
    '-'  :   Token( '-'           , TokenType.minus ),
    '*'  :   Token( '*'           , TokenType.star  ),
    '/'  :   Token( '/'           , TokenType.div   ),
    '('  :   Token( '('           , TokenType.opar  ),
    ')'  :   Token( ')'           , TokenType.cpar  ),
    'pi' :   Token( 3.14159265359 , TokenType.num   ),
    'e'  :   Token( 2.71828182846 , TokenType.num   ),
    'phi':   Token( 1.61803398875 , TokenType.num   ),
    'sen':   Token( 'sen'         , TokenType.id    ),
    'cos':   Token( 'cos'         , TokenType.id    ),
    'tan':   Token( 'tan'         , TokenType.id    ),
    'log':   Token( 'log'         , TokenType.id    ),
    'sqrt':  Token( 'sqrt'        , TokenType.id    )
}

### Lexer

La implementación del lexer es muy similar al de la clase anterior. Por ahora asumiremos que los lexemas relevantes están separados por espacios, por lo que el lexer simplemente debería separar por espacios la cadena de entrada y construir los tokens correspondientes. El lexer deberá incluir un token EOF al final de la secuencia de tokens. Esto resulta conveniente durante el proceso de parsing para evitar manejar el fin de la cadena como un caso extremo.

In [5]:
class HULKLexer(Lexer):

    def tokenize_text(self):
        tokens = []
        text = self.text
        
        for item in text.split():
            if(item.isnumeric()):
                tokens.append(Token(float(item),TokenType.num))
            elif(item in fixed_tokens):
                tokens.append(fixed_tokens[item])
            else:
                tokens.append(item)
        tokens.append(EOF_TOKEN)

        return tokens

### Parser de HULK

Podemos intuir la siguiente gramática de `HULK` a partir de las consideraciones realizadas en la clase anterior:
``` 
E --> T + E | T
T --> F * T | F
F --> ( E ) | n
``` 

Sin embargo, rápidamente podemos notar que un parser recursivo descendente deberá dar "un salto de fe" para decidir qué producción, entre `E --> T + E` y `E --> T`, aplicar desde el inicio. Claro está que como ambos comienzan con `T` se puede aplazar la decisión de cuál producción aplicar hasta terminar de consumir `T`. Para evitar enfrentarnos a esto realizaremos una modificación a la gramática conocida como _eliminación de prefijos comunes_. Para ello, todas las producciones con la misma cabecera que comiencen con el mismo símbolo serán modificadas, con lo cual obtenemos:

```
E --> T X
X --> + T X | - T X | epsilon
T --> F Y
Y --> * F Y | / F Y | epsilon
F --> ( E ) | n
```

Como podremos comprobar con la implementación del parser de `HULK` según la gramática anterior, dicha gramática es _LL(1)_. En todo momento sabremos qué producción aplicar con solo ver el símbolo actual de la cadena.

Para construir el parser según la gramática anterior simplemente extenderemos la clase `Parser` (usando herencia) para incluir un método por cada no terminal de la gramática. En estos métodos deberemos explorar las posibles producciones a aplicar en función del símbolo actual de la cadena (`lookahead`). Según la rama que se decida seguir, invocaremos los métodos correspondientes a los no terminales que aparezcan en la parte derecha de la producción aplicada. Por cada terminal que aparezca en la parte derecha haremos una invocación al método `match` con el tipo del token correspondiente. Este procedimiento se realiza en el orden en que aparezcan los símbolos en la producción. 

> **OJO:** el caso de las producciones con la forma `X --> epsilon` puede ser un tanto especial de seleccionar. Intente descubrir un forma para seleccionar dichas ramas.

Es importante garantizar una invocación al método `error` en caso que ninguna de las producciones (ramas) deba ser aplicada. Esto puede saberse con un análisis manual sobre la gramática.

In [27]:
from anytree import Node , RenderTree

class HULKParser(Parser):
    
    
    def __init__(self):
        self.root = Node('E')

        
    def begin(self):
        self.E(self.root)
        self.match(TokenType.eof)
        self.print_tree(self.root)
        print(self.evaluate_tree(self.root))

    def E(self , node):
        """
        E --> TX
        """
        
        if self.lookahead in (TokenType.num, TokenType.opar):
            self.report('E --> TX')
            T = Node('T', parent = node)
            X = Node('X' , parent = node)
            self.T(T)
            self.X(X)
            
        else:
            self.error('Malformed expression')
        
    def X(self , node):
        """
        X --> +TX | -TX | epsilon
        """
        if(self.lookahead == TokenType.plus):
            self.report('X --> +TX')
            oper = Node('+' , parent = node)
            T = Node('T', parent = node)
            X = Node('X' , parent = node)
            self.match(TokenType.plus)
            self.T(T)
            self.X(X)
        elif(self.lookahead == TokenType.minus):
            self.report('X --> -TX')
            oper = Node('-' , parent = node)
            T = Node('T', parent = node)
            X = Node('X' , parent = node)
            self.match(TokenType.minus)
            self.T(T)
            self.X(X)
        elif(self.lookahead in (TokenType.eof, TokenType.star, TokenType.div , TokenType.num,TokenType.cpar , TokenType.opar)):
            self.report('X --> epsilon')
            epsilon = Node('epsilon' , parent = node)
        else:
            self.error('Malformed expression')

        pass
        
    def T(self , node):
        """
        T --> FY
        """
        if(self.lookahead in (TokenType.num, TokenType.opar , TokenType.star,TokenType.div,TokenType.cpar,TokenType.eof)):
            self.report('T --> FY')
            F  = Node('F' , parent= node)
            Y = Node('Y', parent= node)
            self.F(F)
            self.Y(Y)
            
    def Y(self , node):
        """
        Y --> *FY | /FY | epsilon
        """
        if(self.lookahead == TokenType.star):
            self.report('Y --> *FY')
            oper = Node('*', parent=node)
            F = Node('F' , parent= node)
            Y = Node('Y', parent= node)
            self.match(TokenType.star)
            self.F(F)
            self.Y(Y)
        elif(self.lookahead == TokenType.div):
            self.report('Y --> /FY')
            oper = Node('/', parent=node)
            F = Node('F' , parent= node)
            Y = Node('Y', parent= node)
            self.match(TokenType.div)
            self.F(F)
            self.Y(Y)
        elif(self.lookahead in (TokenType.plus,TokenType.minus,TokenType.num,TokenType.eof,TokenType.cpar, TokenType.opar)):
            self.report('Y --> epsilon')
            epsilon = Node('epsilon', parent= node)
        else:
            self.error('Malformed expression')
        pass
            
    def F(self , node):
        """
        F --> n | (E)
        """
        if(self.lookahead is TokenType.num):
            self.report('F --> n')
            number = Node(self.lookaheadValue , parent= node, tokenType = TokenType.num)
            # print(number.name)
            self.match(TokenType.num)
        else:
            self.report('F --> E')
            bracket1 = Node('(' , parent= node)
            self.match(TokenType.opar)
            E = Node('E' , parent= node)
            braket2 = Node(')' , parent= node)
            self.E(E)
            self.match(TokenType.cpar)
        pass 

    def print_tree (self ,tree):
        for pre , fill , node in RenderTree(tree):
            print("%s%s" % (pre, node.name))

    def evaluate_tree (self ,Tree : Node , value = 0):
        if Tree.name == 'E':
            value = self.evaluate_tree(Tree.children[0])
            return self.evaluate_tree(Tree.children[1],value)
        elif Tree.name == 'X':
            if Tree.children[0].name == '+':
                value += self.evaluate_tree(Tree.children[1])
                return self.evaluate_tree(Tree.children[2],value)
            elif Tree.children[0].name == '-':
                value -= self.evaluate_tree(Tree.children[1])
                return self.evaluate_tree(Tree.children[2],value)
            elif Tree.children[0].name == 'epsilon':
                return value
        elif Tree.name == 'T':
            value = self.evaluate_tree(Tree.children[0])
            return self.evaluate_tree(Tree.children[1], value)
        elif Tree.name == 'Y':
            if Tree.children[0].name == '*':
                value = value * self.evaluate_tree(Tree.children[1],value)
                return self.evaluate_tree(Tree.children[2],value)
            elif Tree.children[0].name == '/':
                value = value / self.evaluate_tree(Tree.children[1],value)
                return self.evaluate_tree(Tree.children[2],value)
            elif Tree.children[0].name == 'epsilon':
                return value
        elif Tree.name == 'F':
            
            try:
                if Tree.children[0].tokenType == TokenType.num:
                    return float(Tree.children[0].name)
            except AttributeError:
                if Tree.children[0].name == '(':
                    return self.evaluate_tree(Tree.children[1])
        else:
            print('else')
            return Tree.name

  



### Pipeline

Cerraremos el _pipeline_ conectando el lexer y el parser, siendo el primero el encargado de preprocesar la cadena de entrada. El parser devuelve un parse izquierdo, con el cual seremos capacez de reconstruir el árbol de derivación y posteriormente evaluar la expresión.

In [29]:
def get_left_parse(text):
    lexer = HULKLexer(text)
    parser = HULKParser()
    return parser.parse(lexer)

assert get_left_parse('5 + 8 * 9') == [  'E --> TX',
                                         'T --> FY',
                                         'F --> n',
                                         'Y --> epsilon',
                                         'X --> +TX',
                                         'T --> FY',
                                         'F --> n',
                                         'Y --> *FY',
                                         'F --> n',
                                         'Y --> epsilon',
                                         'X --> epsilon'  ]

assert get_left_parse('1 - 1 + 1') == [  'E --> TX',
                                         'T --> FY',
                                         'F --> n',
                                         'Y --> epsilon',
                                         'X --> -TX',
                                         'T --> FY',
                                         'F --> n',
                                         'Y --> epsilon',
                                         'X --> +TX',
                                         'T --> FY',
                                         'F --> n',
                                         'Y --> epsilon',
                                         'X --> epsilon'  ]

get_left_parse(' ( 5 + 2 ) + 2') 
get_left_parse(' 2 * ( 5 + 2 )')
get_left_parse('10 / ( 2 + 3 ) * 2')
get_left_parse('1 - 1 + 1')
get_left_parse('5 + 8 * 9')
get_left_parse(' 3 * ( 2 / ( 2 - 3 ) - 1 ) + 5')

E
├── T
│   ├── F
│   │   └── 5.0
│   └── Y
│       └── epsilon
└── X
    ├── +
    ├── T
    │   ├── F
    │   │   └── 8.0
    │   └── Y
    │       ├── *
    │       ├── F
    │       │   └── 9.0
    │       └── Y
    │           └── epsilon
    └── X
        └── epsilon
77.0
E
├── T
│   ├── F
│   │   └── 1.0
│   └── Y
│       └── epsilon
└── X
    ├── -
    ├── T
    │   ├── F
    │   │   └── 1.0
    │   └── Y
    │       └── epsilon
    └── X
        ├── +
        ├── T
        │   ├── F
        │   │   └── 1.0
        │   └── Y
        │       └── epsilon
        └── X
            └── epsilon
1.0
E
├── T
│   ├── F
│   │   ├── (
│   │   ├── E
│   │   │   ├── T
│   │   │   │   ├── F
│   │   │   │   │   └── 5.0
│   │   │   │   └── Y
│   │   │   │       └── epsilon
│   │   │   └── X
│   │   │       ├── +
│   │   │       ├── T
│   │   │       │   ├── F
│   │   │       │   │   └── 2.0
│   │   │       │   └── Y
│   │   │       │       └── epsilon
│   │   │       └── X
│   │   │          

['E --> TX',
 'T --> FY',
 'F --> n',
 'Y --> *FY',
 'F --> E',
 'E --> TX',
 'T --> FY',
 'F --> n',
 'Y --> /FY',
 'F --> E',
 'E --> TX',
 'T --> FY',
 'F --> n',
 'Y --> epsilon',
 'X --> -TX',
 'T --> FY',
 'F --> n',
 'Y --> epsilon',
 'X --> epsilon',
 'Y --> epsilon',
 'X --> -TX',
 'T --> FY',
 'F --> n',
 'Y --> epsilon',
 'X --> epsilon',
 'Y --> epsilon',
 'X --> +TX',
 'T --> FY',
 'F --> n',
 'Y --> epsilon',
 'X --> epsilon']

## Adicionando funciones elementales

Agreguemos funciones elementales `sin`, `cos`, `tan`, `log`, `sqrt`, etc. El llamado a funciones se hará en notación prefija, comenzando por el nombre de la función y seguido, entre paréntesis, por los argumentos, que estarán separados entre sí por _comas_.

In [8]:
import math

functions = {
    'sin': lambda x: math.sin(x),
    'cos': lambda x: math.cos(x),
    'tan': lambda x: math.tan(x),
    'log': lambda x,y: math.log(x, y),
    'sqrt': lambda x: math.sqrt(x),
}

### Reconstrucción del árbol de derivación y evaluación

- Realice las modificaciones pertinentes para que las producciones reportadas por el parser nos permitan reconstruir el árbol de derivación. Note que la implementación actual trabaja con `str` y se desechan los tokens (que son los contenedores de los lexemas).
- Utilice el parse izquierdo y/o árbol de derivación para evaluar la expresión. Note que la estructura de la gramática causa que los operadores (+, -, \*, /) asocien hacia la derecha, lo cual conlleva problemas si se evalúa recursivamente sin considerar tal característica.